In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

from __future__ import print_function

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

In [2]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

from WordVector import WordVector

In [3]:
import data_helper
_,_,relation = data_helper.get_triplet()
relation[:5]

['cause', 'cause', 'cause', 'cause', 'cause']

In [4]:
relation = list(set(relation))
relation[:5]

['infection', 'obtain', 'use', 'appear', 'colonise']

In [5]:
wv = WordVector(relation, dim=3)

Loading a pre-trained model...
Load success!


In [6]:
wv.word[:5]

['infection', 'obtain', 'use', 'appear', 'colonise']

In [7]:
wv.vector[0]

array([ 4.6460371 ,  3.42395353, -2.84098554], dtype=float32)

# Silhouette Coefficient Analysis

In [8]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
colors=["rgb(255,0,0)", "rgb(0,255,0)", "rgb(0,0,255)", "rgb(255,255,0)", "rgb(255,0,255)", 
        "rgb(0,255,255)", "rgb(127,0,255)", "rgb(255,128,0)","rgb(255,0,127)", "rgb(128,128,128)"]

In [9]:
figures = []

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig = tools.make_subplots(rows=1, cols=2,
                              print_grid=False,
                              specs=[[{'is_3d': False}, {'is_3d': True}]])

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    fig['layout']['xaxis1'].update(title='The silhouette coefficient values', range=[-0.1, 1])
   
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    fig['layout']['yaxis1'].update(title='Cluster label',
                                   showticklabels=False,
                                   range=[0, len(wv.vector) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters)
    cluster_labels = clusterer.fit_predict(wv.vector)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(wv.vector, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(wv.vector, cluster_labels)
    y_lower = 10
    
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        # colors = cm.spectral(cluster_labels.astype(float) / n_clusters)

        filled_area = go.Scatter(y=np.arange(y_lower, y_upper),
                                 x=ith_cluster_silhouette_values,
                                 mode='lines',
                                 showlegend=False,
                                 line=dict(width=0.5,
                                          color=colors[i]),
                                 fill='tozerox')
        fig.append_trace(filled_area, 1, 1)
        
        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples
        

    # The vertical line for average silhouette score of all the values
    axis_line = go.Scatter(x=[silhouette_avg],
                           y=[0, len(wv.vector) + (n_clusters + 1) * 10],
                           showlegend=False,
                           mode='lines',
                           line=dict(color="red", dash='dash',
                                     width =1) )

    fig.append_trace(axis_line, 1, 1)
    
    
    
    
    
    
    # 2nd Plot showing the actual clusters formed
    
    labeled_vec = list(zip(wv.vector, cluster_labels))
    # Labeling the clusters
    centers_ = clusterer.cluster_centers_
    for i in range(n_clusters):
        ith_cluster_vector = np.array([v for (v,c) in labeled_vec if c == i])
        # colors = matplotlib.colors.colorConverter.to_rgb(cm.spectral(float(i) / n_clusters))
        # colors = 'rgb'+str(colors)
        
        clusters = go.Scatter3d(x=ith_cluster_vector[:, 0], 
                                y=ith_cluster_vector[:, 1], 
                                z=ith_cluster_vector[:, 2],
                                showlegend=False,
                                mode='markers',
                                marker=dict(
                                    color=colors[i],
                                    size=3,
                                    line=dict(
                                        width=0.5
                                    )
                                )
                               )
        fig.append_trace(clusters, 1, 2)
        
        # Draw white circles at cluster centers
        centers = go.Scatter3d(x=centers_[i][0], 
                               y=centers_[i][1],
                               z=centers_[i][2],
                               showlegend=False,
                               mode='markers',
                               marker=dict(
                                   color=colors[i], size=5,
                                   line=dict(color='black', width=1))
                               )

        fig.append_trace(centers, 1, 2)
    
                      
        
        
        

    fig['layout'].update(title="Silhouette analysis for KMeans clustering on sample data "
                         "with n_clusters = %d" % n_clusters)
    
    figures.append(fig)

For n_clusters = 2 The average silhouette_score is : 0.358086
For n_clusters = 3 The average silhouette_score is : 0.342406
For n_clusters = 4 The average silhouette_score is : 0.409304
For n_clusters = 5 The average silhouette_score is : 0.406962
For n_clusters = 6 The average silhouette_score is : 0.405057
For n_clusters = 7 The average silhouette_score is : 0.405453
For n_clusters = 8 The average silhouette_score is : 0.408889
For n_clusters = 9 The average silhouette_score is : 0.395544
For n_clusters = 10 The average silhouette_score is : 0.41415


In [10]:
py.iplot(figures[0])

In [11]:
py.iplot(figures[1])

In [12]:
py.iplot(figures[2])